In [ ]:
!pip install xdggs
!pip install healpy
!pip install -U zarr==2.18.4
!pip install numcodecs==0.13.1
!pip install -U xarray

In [10]:
import warnings
import healpy as hp
#import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import xdggs
#import fsspec

warnings.filterwarnings("ignore")
xr.set_options(display_max_rows=27)

In [7]:
url="https://data-fair2adapt.ifremer.fr/odysea/test/F_c13_all_nest.zarr"
ds_tree=xr.open_datatree(url, engine='zarr')


In [3]:

def create_arrow_table(polygons, arr, coords=None):
    from arro3.core import Array, ChunkedArray, Schema, Table

    if coords is None:
        coords = ["latitude", "longitude"]

    array = Array.from_arrow(polygons)
    name = arr.name or "data"
    arrow_arrays = {
        "geometry": array,
        "cell_ids": ChunkedArray([Array.from_numpy(arr.coords["cell_ids"])]),
        name: ChunkedArray([Array.from_numpy(arr.data)]),
    } | {
        coord: ChunkedArray([Array.from_numpy(arr.coords[coord].data)])
        for coord in coords
        if coord in arr.coords
    }

    fields = [array.field.with_name(name) for name, array in arrow_arrays.items()]
    schema = Schema(fields)

    return Table.from_arrays(list(arrow_arrays.values()), schema=schema)


def normalize(var, center=None):
    from matplotlib.colors import CenteredNorm, Normalize

    if center is None:
        vmin = var.min(skipna=True)
        vmax = var.max(skipna=True)
        normalizer = Normalize(vmin=vmin, vmax=vmax)
    else:
        halfrange = np.abs(var - center).max(skipna=True)
        normalizer = CenteredNorm(vcenter=center, halfrange=halfrange)

    return normalizer(var.data)


def exploire_layer(
    arr,
    cell_dim="cells",
    cmap="viridis",
    center=None,
    alpha=None,
):
    from lonboard import SolidPolygonLayer
    from lonboard.colormap import apply_continuous_cmap
    from matplotlib import colormaps

    if len(arr.dims) != 1 or cell_dim not in arr.dims:
        raise ValueError(
            f"exploration only works with a single dimension ('{cell_dim}')"
        )

    cell_ids = arr.dggs.coord.data
    grid_info = arr.dggs.grid_info

    polygons = grid_info.cell_boundaries(cell_ids, backend="geoarrow")

    normalized_data = normalize(arr.variable, center=center)

    colormap = colormaps[cmap]
    colors = apply_continuous_cmap(normalized_data, colormap, alpha=alpha)

    table = create_arrow_table(polygons, arr)
    layer = SolidPolygonLayer(table=table, filled=True, get_fill_color=colors)

    return layer

In [20]:
import lonboard
dtotal = ds_tree["9"].ds
dtotal
ds_9=(np.sqrt(
    dtotal.u_model.where(lambda x : x!=hp.UNSEEN)**2
    + dtotal.v_model.where(lambda x : x!=hp.UNSEEN)**2)
    #.dggs.sel_latlon(longitude=full_lon, latitude=full_lat)
    .mean(dim="orbit").compute()#.dggs.explore(alpha=0.8)
   ).dropna(dim='cells').pipe(xdggs.decode)
ds_9.dggs.explore(alpha=0.8)

Map(layers=[SolidPolygonLayer(filled=True, get_fill_color=<pyarrow.lib.FixedSizeListArray object at 0x7acace8e…

In [21]:
import lonboard
dtotal = ds_tree["10"].ds
dtotal
ds_10=(np.sqrt(
    dtotal.u_model.where(lambda x : x!=hp.UNSEEN)**2
    + dtotal.v_model.where(lambda x : x!=hp.UNSEEN)**2)
    #.dggs.sel_latlon(longitude=full_lon, latitude=full_lat)
    .mean(dim="orbit").compute()#.dggs.explore(alpha=0.8)
   ).dropna(dim='cells').pipe(xdggs.decode)
#ds_10.dggs.explore(alpha=0.8)

In [22]:
lonboard.Map(
    [
        exploire_layer(
            ds_9,
            alpha=1,
        ),
        exploire_layer(ds_10, alpha=1),
    ]
)

Map(layers=[SolidPolygonLayer(filled=True, get_fill_color=<pyarrow.lib.FixedSizeListArray object at 0x7acaccff…